In [ ]:
#Install Module
!pip install ortools --use-deprecated=legacy-resolver

In [ ]:
#import module
from ortools.sat.python import cp_model
def main() -> None :
    num_nurses = 8 #Số nhân viên
    num_shifts = 2 #Số lượng ca : sáng-tối
    num_days = 28 #Số lượng ngày
    nurses_name_list = ['An', 'Bình', 'Châu', 'Dương', 'Linh', 'Kiệt', 'Giang', 'Hiếu'] #Tên các y tá
    shifts_name = ['S', 'T'] #Tên các ca trực

    # Khởi tạo các iterables
    all_nurses = range(num_nurses)
    full_time_nurses = range(4)
    part_time_nurses = range(4,8)
    all_shifts = range(num_shifts)
    all_days = range(num_days)


    # Initialize model
    model = cp_model.CpModel()
    # Model chính
    shifts = {}
    for n in all_nurses:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d, s)] = model.new_bool_var(f"shift_n{n}_d{d}_s{s}")
# Các điều kiện ban đầu :
    # Mỗi y tá làm tối đa một ca mỗi ngày
    for n in all_nurses:
        for d in all_days:
            model.add_at_most_one(shifts[(n, d, s)] for s in all_shifts)

# Hard-constraints
    # Mỗi nhân viên full-time làm tổng cộng tối đa 18 ca, nhân viên part-time làm tối đa 10 ca
    for n in full_time_nurses:
        shifts_fulltime_worked = []
        for d in all_days:
            for s in all_shifts:
                shifts_fulltime_worked.append(shifts[(n, d, s)])
        model.add(sum(shifts_fulltime_worked) == 18)

    for n in part_time_nurses:
        shifts_parttime_worked = []
        for d in all_days:
            for s in all_shifts:
                shifts_parttime_worked.append(shifts[(n, d, s)])
        model.add(sum(shifts_parttime_worked) == 10)


    # Mỗi ngày có đúng 3 nhân viên trực ca sáng, 1 nhân viên ca tối
    for d in all_days :
        day_shifts = []
        night_shifts = []
        for n in all_nurses :
            day_shifts.append(shifts[(n, d, 0)])
            night_shifts.append(shifts[(n, d, 1)])
        model.add(sum(day_shifts) == 3)
        model.add(sum(night_shifts) == 1)

# Soft-constraints :
    # Mỗi nhân viên full-time mong muốn làm từ 4-6 ca mỗi tuần, tương tự 2-3 ca với nhân viên part-time

    for w in full_time_nurses:
        for n in full_time_nurses:  # First 4 nurses
            fulltime_weekly_shifts = sum(shifts[(n, d, s)]
                                for d in range(w * 7, (w + 1) * 7)
                                for s in range(2))
            model.Add(fulltime_weekly_shifts >= 4)
            model.Add(fulltime_weekly_shifts <= 5)
        for n in part_time_nurses:  # Last 4 nurses
            parttime_weekly_shifts = sum(shifts[(n, d, s)]
                                for d in range(w * 7, (w + 1) * 7)
                                for s in range(2))
            model.Add(parttime_weekly_shifts >= 2)
            model.Add(parttime_weekly_shifts <= 3)


    # Mỗi nhân viên full-time muốn một chuỗi ngày làm việc kéo dài từ 4-6 ngày
    # Tương tự 2-3 ngày với nhân viên part-time

    # Full-time nurses :
    for n in full_time_nurses:

        for d in range(num_days-6):
            model.Add(sum(shifts[(n, j, s)] for j in range(d,d+7)
                                            for s in range(2)) <= 6)
        #Case day 1
        condition_case1_1_1 = model.NewBoolVar('')
        condition_case1_1_2 = model.NewBoolVar('')
        model.Add(sum(shifts[(n, 0, s)] for s in range(2)) == 0).OnlyEnforceIf(condition_case1_1_1)
        model.Add(sum(shifts[(n, j, s)] for j in range(4) for s in range(2)) == 4).OnlyEnforceIf(condition_case1_1_2)
        model.AddBoolOr([condition_case1_1_1, condition_case1_1_2])
        #Case day 2-25
        for d in range(1, num_days-3):
            condition_1_1 = model.NewBoolVar('')
            condition_1_2 = model.NewBoolVar('')
            condition_1_3 = model.NewBoolVar('')
            model.Add(sum(shifts[(n, d, s)] for s in range(2)) == 0).OnlyEnforceIf(condition_1_1)
            model.Add(sum(shifts[(n, d-1, s)] for s in range(2)) == 1).OnlyEnforceIf(condition_1_2)
            model.Add(sum(shifts[(n, j, s)] for j in range(d,d+4) for s in range(2)) == 4).OnlyEnforceIf(condition_1_3)
            model.AddBoolOr([condition_1_1, condition_1_2, condition_1_3])
        #Case day 26
        condition_case26_1_1 = model.NewBoolVar('')
        condition_case26_1_2 = model.NewBoolVar('')
        condition_case26_1_3 = model.NewBoolVar('')
        model.Add(sum(shifts[(n, 25, s)] for s in range(2)) == 0).OnlyEnforceIf(condition_case26_1_1)
        model.Add(sum(shifts[(n, 24, s)] for s in range(2)) == 1).OnlyEnforceIf(condition_case26_1_2)
        model.Add(sum(shifts[(n, j, s)] for j in range(25,28) for s in range(2)) == 3).OnlyEnforceIf(condition_case26_1_3)
        model.AddBoolOr([condition_case26_1_1,condition_case26_1_2, condition_case26_1_3])
        #Case day 27
        condition_case27_1_1 = model.NewBoolVar('')
        condition_case27_1_2 = model.NewBoolVar('')
        condition_case27_1_3 = model.NewBoolVar('')
        model.Add(sum(shifts[(n, 26, s)] for s in range(2)) == 0).OnlyEnforceIf(condition_case27_1_1)
        model.Add(sum(shifts[(n, 25, s)] for s in range(2)) == 1).OnlyEnforceIf(condition_case27_1_2)
        model.Add(sum(shifts[(n, j, s)] for j in range(26,28) for s in range(2)) == 2).OnlyEnforceIf(condition_case27_1_3)
        model.AddBoolOr([condition_case27_1_1,condition_case27_1_2, condition_case27_1_3])

    # Part-time nurses :
    for n in part_time_nurses:

        for d in range(num_days-3):
            model.Add(sum(shifts[(n, j, s)]
                          for j in range(d,d+4)
                          for s in range(2)) <= 3)
        #Case day 2-25
        for d in range(1, num_days-1):
            condition_2_1 = model.NewBoolVar('')
            condition_2_2 = model.NewBoolVar('')
            condition_2_3 = model.NewBoolVar('')
            model.Add(sum(shifts[(n, d, s)]
                          for s in range(2)) == 0
                      ).OnlyEnforceIf(condition_2_1)

            model.Add(sum(shifts[(n, d-1, s)]
                          for s in range(2)) == 1
                      ).OnlyEnforceIf(condition_2_2)

            model.Add(sum(shifts[(n, j, s)]
                          for j in range(d,d+2) for s in range(2)) == 2
                      ).OnlyEnforceIf(condition_2_3)

            model.AddBoolOr([condition_2_1, condition_2_2, condition_2_3])
        #Case day 1
        condition_case1_2_1 = model.NewBoolVar('')
        condition_case1_2_2 = model.NewBoolVar('')
        model.Add(sum(shifts[(n, 0, s)]
                      for s in range(2)) == 0
                  ).OnlyEnforceIf(condition_case1_2_1)

        model.Add(sum(shifts[(n, j, s)]
                      for j in range(2)
                      for s in range(2)) == 2
                  ).OnlyEnforceIf(condition_case1_2_2)

        model.AddBoolOr([condition_case1_2_1, condition_case1_2_2])
        #Case day 27
        condition_case27_2_1 = model.NewBoolVar('')
        condition_case27_2_2 = model.NewBoolVar('')
        condition_case27_2_3 = model.NewBoolVar('')
        model.Add(sum(shifts[(n, 26, s)] for s in range(2)) == 0
                     ).OnlyEnforceIf(condition_case27_2_1)
        model.Add(sum(shifts[(n, 25, s)] for s in range(2)) == 1
                      ).OnlyEnforceIf(condition_case27_2_2)
        model.Add(sum(shifts[(n, j, s)]
            for j in range(26,28) for s in range(2)) == 2
                      ).OnlyEnforceIf(condition_case27_2_3)
        model.AddBoolOr([condition_case27_2_1,condition_case27_2_2, condition_case27_2_3])




    # Tạo Solver của model
    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    # Duyệt qua tất cả các trường hợp
    solver.parameters.enumerate_all_solutions = True

    class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
        """Print intermediate solutions."""

        def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self._shifts = shifts
            self._num_nurses = num_nurses
            self._num_days = num_days
            self._num_shifts = num_shifts
            self._solution_count = 0
            self._solution_limit = limit

        def on_solution_callback(self) :
          self._solution_count += 1
          print(f"Solution {self._solution_count}")
          print("Ngày   ", end='')
          for day in range(28):
              print(f"{day + 1:02d} ", end='')
          print()
          for i, nurse_name in enumerate(nurses_name_list):
              print(f"{nurse_name:7}", end='')
              for day in range(28):
                  is_working = False
                  for s in range(2) :
                      if self.value(self._shifts[(i, day, s)]) :
                          is_working = True
                          print(f"{shifts_name[s]:2} ", end = '')
                  if not is_working :
                      print ('   ', end = '')
              print()

          if self._solution_count >= self._solution_limit:
              print(f"Stop search after {self._solution_limit} solutions")
              self.stop_search()
        def solutionCount(self):
            return self._solution_count

    # In ra một lời giải thỏa mãn
    solution_limit = 1
    solution_printer = NursesPartialSolutionPrinter(
        shifts, num_nurses, num_days, num_shifts, solution_limit
    )

    solver.solve(model, solution_printer)
    print(f"Solution count: {solution_printer.solutionCount()}")
main()

Solution 1
Ngày   01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
An     S  S  S  S        S  S  S  S           S  S  S  S        S  S  S  S  S  T           
Bình   S  S  T  S                 S  S  S  T        S  T  T  S  S        S  S  S  T     T  
Châu         S  T  S  T        S  S  S  S  S           S  S  T  T           S  S  S  S  S  
Dương  T  T  S  S  S        S  T  T  S              S  S  S  S           S  T  S  S  S     
Linh               S  S     T  S              S  T                 S  T           S  S     
Kiệt               T  S  T              S  S              S  S  S                    T  S  
Giang                 S  S           T  T     S  S  T              S  S  T                 
Hiếu   S  S              S  S              S  T  S                 T  S                 S  
Stop search after 1 solutions
Solution count: 1
